
# Projet 2

In [1]:
# import des modules usuels
import numpy as np
import pandas as pd
import json
from shapely.geometry import Polygon

# options d'affichage
pd.set_option("display.min_rows", 16)

In [2]:
# chargement et traitement des données
geo = pd.read_csv("correspondance-code-insee-code-postal.csv",
                   sep=';',
                   usecols=range(11),
                   index_col="Code INSEE")
geo = geo.sort_index()
geo[['Latitude', 'Longitude']] = geo["geo_point_2d"].str.extract("(.*), (.*)").astype(float)
geo.head()

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude
Code INSEE,,,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,1565.0,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273...",46.153426,4.926114
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,912.0,0.2,"46.0091878776, 5.42801696363","{""type"": ""Polygon"", ""coordinates"": [[[5.430089...",46.009188,5.428017
01004,01500,AMBERIEU-EN-BUGEY,AIN,RHONE-ALPES,Chef-lieu canton,379.0,2448.0,13.4,"45.9608475114, 5.3729257777","{""type"": ""Polygon"", ""coordinates"": [[[5.386190...",45.960848,5.372926
01005,01330,AMBERIEUX-EN-DOMBES,AIN,RHONE-ALPES,Commune simple,290.0,1605.0,1.6,"45.9961799872, 4.91227250796","{""type"": ""Polygon"", ""coordinates"": [[[4.895580...",45.996180,4.912273
01006,01300,AMBLEON,AIN,RHONE-ALPES,Commune simple,589.0,602.0,0.1,"45.7494989044, 5.59432017366","{""type"": ""Polygon"", ""coordinates"": [[[5.614854...",45.749499,5.594320


**Projet maison n° 1**

- Ajoutez une colonne 'CP Ville' avec le Code postal + un espace + et le nom de la Ville.
- Ecrivez une fonction qui détermine la commune la plus proche d'un point à partir de sa latitude et sa longitude.
- Ajoutez une fonction de conversion pour pouvoir utiliser la première fonction avec un GPS (degrés, minutes, secondes).

In [3]:
geo['CP Ville'] = geo['Code Postal'] + " " + geo['Commune']

In [30]:
# fonction recherche de ville
def ville(lat, long):
    temp = geo
    temp['dist'] = np.sqrt(((lat - geo.Latitude)**2 + (long - geo.Longitude)**2))
    print(temp[temp['dist'].idxmin()].Commune.values[0])

In [31]:
# conversion degrés, minutes, secondes => décimal
def dms2dec(deg, mn, sec):
    return deg + mn / 60 + sec / 3600

In [32]:
# on applique la fonction à une coordonnée tirée au hasard
# init du random
np.random.seed(0)

a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
lat = (b - a) * np.random.random() + a
a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
long = (b - a) * np.random.random() + a

ville(lat, long)

KeyError: '71044'

In [7]:
# à partir de coordonnées GPS précises
ville(dms2dec(48, 42, 52), dms2dec(2, 14, 45))

'PALAISEAU'

**Projet maison n° 2**

La colonne "geo_shape" comporte des chaines de catactères au format JSON. Elles représentent les formes géométriques des communes qui sont soit des polygones soit composées de plusieurs polygones.

- Utiliser la librairie Python json pour parser la colonne "geo_shape".

In [8]:
geo_shape_json = pd.json_normalize(geo.geo_shape.apply(json.loads))
geo_shape_json = geo_shape_json.set_index(geo.index)

In [9]:
geo_shape_json.head()

,type,coordinates
Code INSEE,,
01001,Polygon,"[[[4.92627342692396, 46.1200517234555], [4.921..."
01002,Polygon,"[[[5.430089291292173, 45.98275997279625], [5.4..."
01004,Polygon,"[[[5.386190824025674, 45.930927620632], [5.374..."
01005,Polygon,"[[[4.895580369408855, 45.97991618124606], [4.8..."
01006,Polygon,"[[[5.614854625053272, 45.734638855121595], [5...."


- Donner le décompte des valeurs accédées avec la clé "type".



In [10]:
geo_shape_json.type.value_counts()

Polygon         36670
MultiPolygon       72
Name: type, dtype: int64

- Donner le décompte des longueurs des listes accédées avec la clé "coordinates".



In [11]:
list(geo_shape_json['coordinates'].apply(len).unique())

[1, 2, 3, 4]

In [12]:
geo_shape_json['coordinates'].apply(len).value_counts()

1    36660
2       80
4        1
3        1
Name: coordinates, dtype: int64

- Quelle commune est la plus complexe géométriquement ?



In [13]:
index_complex = geo_shape_json[geo_shape_json['coordinates'].apply(len) == 4].index
print(geo.loc[index_complex, 'Commune'].values[0])

HYERES


- Quelle commune est la seconde la plus complexe géométriquement ?



In [17]:
index_second_complex = geo_shape_json[geo_shape_json['coordinates'].apply(len) == 3].index
print(geo.loc[index_second_complex, 'Commune'].values[0])

VAL-DE-REUIL


- Quelles sont les villes qui sont de type "Polygon" mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2 ?



In [15]:
index_type = geo_shape_json[(geo_shape_json['type'] == "Polygon") & (geo_shape_json['coordinates'].apply(len) == 2)].index

for ville in geo.Commune.loc[index_type]:
    print(ville)

COYOLLES
EYGLIERS
LA BASTIDE-DE-SEROU
PLOUBALAY
SARTENE
CALENZANA
LANGOGNE
SUNDHOUSE
EGUISHEIM
BEAUVERNOIS


- Pour ces villes vérifier que le premier polygone contient bien le second (enclave). NB : installer la librairie shapely, utiliser la classe Polygon de shapely.geometry. Sur Windows shapely peut nécessiter d'installer la dll "geos_c.dll" dans le répertoire "Library/bin" de votre environnement Python.

In [20]:
bool_value = True 
for idx in index_type:
    coordinates = geo_shape_json.loc[idx, 'coordinates']
    polygon1 = Polygon(coordinates[0])
    polygon2 = Polygon(coordinates[1])
    bool_value = bool_value and polygon1.contains(polygon2)
print("Ces villes vérifient que le premier polygone contient bien le second : ",bool_value)

Ces villes vérifient que le premier polygone contient bien le second :  True
